In [1]:
import numpy as np
import pandas as pd
import re
import multiprocessing as mp
from tqdm.auto import tqdm
from tqdm.contrib import tzip
import spacy
import pickle
from string import punctuation
from spacy.lang.en import stop_words

/usr/local/lib/python3.11/dist-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2024-10-22 18:43:38.056142: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-22 18:43:38.056201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-22 18:43:38.057267: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-22 18:43:38.063131: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructio

In [2]:
class TextProcessor():
    def __init__(self, kn_base='en_core_web_sm'):
        self.nlp = spacy.load(kn_base)
        self.stop_wrods = stop_words.STOP_WORDS | set(punctuation) | set('123456789')
        
    def prepare_text(self, s):
        s = s.lower().replace('_',' ').strip()
        for p in punctuation:
            s = s.replace(p,' ')
        return re.sub(" +", " ",s)
    
    def precess_text(self,s):
        s = self.prepare_text(s)
        doc = self.nlp(s)
        lemms = [token.lemma_ for token in doc]
        lemms = [token for token in lemms if token not in self.stop_wrods]
        return lemms
    
    def process_batch(self,texts,verbose=True):
        iterator = tqdm(texts) if verbose else texts
        pool = mp.Pool(mp.cpu_count())
        prep_texts = pool.map(self.precess_text,iterator)
        return prep_texts

In [3]:
class EntityLinlker():
    def __init__(self, kg, text_proc, prepared2concept):
        self.kg = kg
        self.text_proc = text_proc
        self.p2c = prepared2concept
        
    def link_sentence(self,s):
        candidates = []
        s = self.text_proc.precess_text(s)
        s_st = set(s)
        for i,k in enumerate(self.kg):
            if len(k) == 0:
                continue
            elif len(k) == 1:
                if k[0] in s_st:
                    candidates.append(i)
            else:
                if all(x in s_st for x in k):
                    idxes = [s.index(x) for x in k]
                    if all(idxes[j+1] - idxes[j] == 1 for j in range(len(k)-1)):
                           candidates.append(i)
        return candidates
    
    def batch_linking(self,texts,verbose=True):
        iterator = tqdm(texts) if verbose else texts
        pool = mp.Pool(mp.cpu_count())
        prep_texts = pool.map(self.link_sentence,iterator)
        return prep_texts

In [4]:
processor = TextProcessor()

In [5]:
graph_state = pickle.load(open('graph.pkl','rb'))

In [6]:
concepts = graph_state['concepts_maper'].keys()
concepts = [(x if type(x) == str else '') for x in concepts]

In [7]:
prepared_concepts =  pickle.load(open('conceprs_enc.pkl','rb'))#processor.process_batch(concepts)

In [8]:
concept2prepared = dict(zip(concepts,prepared_concepts))

In [9]:
def texual_acc(s1,s2): # like charackter bleu@1 score
    return sum(i == j for i,j in zip(s1,s2)) / (max(len(s1),len(s2)) + 1e-6)

def prep_text(s):
    return s.lower().strip().replace('_', ' ')

def rank_func(s1,texts):
    s1 = prep_text(s1)
    scores = []
    for s in texts:
        scores.append(texual_acc(s1,prep_text(s)))
    return texts[np.argmax(scores)]

In [10]:
df_cp = pd.DataFrame(zip(concepts,prepared_concepts))
df_cp[1] = df_cp[1].apply(lambda x: ' '.join(x))
df_cp_grp = df_cp.groupby(1)[0].agg(list)

In [11]:
ranked_nodes = [rank_func(y,x) for x,y in tzip(df_cp_grp.tolist(), df_cp_grp.index.tolist())]

  0%|          | 0/656261 [00:00<?, ?it/s]

In [12]:
prepared2concept = dict(zip(df_cp_grp.index.tolist(),ranked_nodes))

In [13]:
def parse_dialog(sentences):
    return '; '.join(sentences)

def parse_request(x):
    return f"{x['dialog']} {x['question']} {x['choice_0']} {x['choice_1']}  {x['choice_2']}"
    

def make_df(path, is_test=False):
    data = pd.read_json(path,lines=True)
    df = []
    for i in range(len(data)):
        row = data.iloc[i]
        dialog = parse_dialog(row[0])
        for ques in row[1]:
            df.append({
                'dialog_id': i,
                'dialog': dialog,
                'question': ques['question'],
                'choice_0': ques['choice'][0],
                'choice_1': ques['choice'][1],
                'choice_2': ques['choice'][2],
                'labels': -100 if is_test else ques['choice'].index(ques['answer'])
            })
    return pd.DataFrame(df)

In [14]:
train_data = make_df('data/train.statement.jsonl')
vall_data = make_df('data/dev.statement.jsonl')
test_data = make_df('data/test.statement.jsonl',is_test=True)

In [15]:
train_data['request'] = train_data.apply(parse_request,axis=1)
vall_data['request'] = vall_data.apply(parse_request,axis=1)
test_data['request'] = test_data.apply(parse_request,axis=1)

In [16]:
linker = EntityLinlker(prepared_concepts, processor, prepared2concept)

In [17]:
train_data['candidates'] = linker.batch_linking(train_data['request'].tolist())

  0%|          | 0/7307 [00:00<?, ?it/s]

In [18]:
vall_data['candidates'] = linker.batch_linking(vall_data['request'].tolist())

  0%|          | 0/813 [00:00<?, ?it/s]

In [19]:
test_data['candidates'] = linker.batch_linking(test_data['request'].tolist())

  0%|          | 0/2077 [00:00<?, ?it/s]

In [20]:
def get_concepts(ids):
    ids = [prepared_concepts[i] for i in ids]
    ids = [' '.join(x) for x in ids]
    return [prepared2concept[i] for i in ids]

In [21]:
test_data['candidates'] = test_data['candidates'].apply(get_concepts)
train_data['candidates'] = train_data['candidates'].apply(get_concepts)
vall_data['candidates'] = vall_data['candidates'].apply(get_concepts)

In [22]:
idf_matrix = pd.Series(
    np.concatenate(test_data['candidates'].tolist() + train_data['candidates'].tolist() + vall_data['candidates'].tolist())
).value_counts()

In [23]:
idf_matrix = idf_matrix.to_dict()

In [24]:
def rank_by_idf(texts,topk=5):
    idx = np.unique(texts, return_index=True)[1]
    texts = [texts[i] for i in sorted(idx)]
    idfs = [idf_matrix[x] for x in texts]
    return [texts[i] for i in np.argsort(idfs)[:topk]]

In [25]:
test_data['candidates_rank'] = test_data['candidates'].apply(rank_by_idf)
train_data['candidates_rank'] = train_data['candidates'].apply(rank_by_idf)
vall_data['candidates_rank'] = vall_data['candidates'].apply(rank_by_idf)

In [26]:
test_data['candidates_rank'].to_csv('./prep_data/test_rank_by_ids.csv',index=False)
train_data['candidates_rank'].to_csv('./prep_data/train_rank_by_ids.csv',index=False)
vall_data['candidates_rank'].to_csv('./prep_data/vall_rank_by_ids.csv',index=False)

In [25]:
with open('idf_dream.pkl','wb') as f:
    pickle.dump(idf_matrix,f)